In [1]:
!pip install accelerate \
    protobuf \
    sentencepiece \
    torch \
    git+https://github.com/huggingface/transformers \
    huggingface_hub

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-tpul0wch
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-tpul0wch
  Resolved https://github.com/huggingface/transformers to commit 25245ec26dc29bcf6102e1b4ddd0dfd02e720cf5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu1

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [5]:
from huggingface_hub import login
login()

In [7]:
if torch.cuda.is_available():
    model_id = "NousResearch/Llama-2-7b-chat-hf"
    # uncomment below to use finetune
    # model_id = "calvinlaughlin/llama-2-7b-legobuilder"
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
    model.cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [12]:
def chat_with_llama(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to('cuda')
    output = model.generate(input_ids, max_length=2048, num_beams=2, no_repeat_ngram_size=2)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

while True:
    title = input("Prompt title: ")
    prompt = input("You: ")
    response = chat_with_llama(prompt)
    print("Llama:", response)

    # Save the response to a text file
    filename = f"llama2_{title}.txt"
    with open(filename, 'w') as file:
        file.write(response)

    print(f"Response saved to {filename}")

Prompt title: house_2
You: Provide step-by-step instructions for a LEGO build with this description: a countryside manor. Make sure that each step is doable in the real world and that each piece is a real lego piece, include the part number.Firstly, state every lego piece that you will need in the build. Then, create a small story about the build. Finally, create the comprehensive step by step guide of the builds.
Llama: Provide step-by-step instructions for a LEGO build with this description: a countryside manor. Make sure that each step is doable in the real world and that each piece is a real lego piece, include the part number.Firstly, state every lego piece that you will need in the build. Then, create a small story about the build. Finally, create the comprehensive step by step guide of the builds.
LEGO Pieces Needed:

1. 6077 - 1x2 Plate with 2 Holes (2)
2.10213 - Window Frame (1) 
3.6126 - Door Frame with Hinge (3) (Note: These are the only exclusive pieces needed for this buil

KeyboardInterrupt: Interrupted by user